In [6]:
import flywheel
import pandas as pd
fw = flywheel.Client()
gear = fw.lookup('gears/wscore-ct-heatmap/0.2.1_0.2.0')
collection_id = '6184053063bd671113926cc7'

In [7]:
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [8]:
df = pd.read_csv("../PVS_subjects.csv")

In [10]:
# Create a data dict:
data_dict = {'subject':[],'session':[],'run':[],'ants':[]}

# Iterate over sessions
for session in sessions:
    sub_label = session.subject.label
    ses_label = session.label
    print(f'Running neuroprint on {ses_label}...')
    run = 'already_run'
    ants = 'na'
    for analysis in session.analyses:     
        # probably only need to run on most recent batch. unless there were subjects with old ANTs run but no wscore run...
        if 'antsct_2021-11-04_WT_PVS' in analysis.label and analysis.job['state']=='complete' and analysis.files:
            # get antsct zip file
            zip_file = [f for f in analysis.files if f.name.endswith('.zip')][0]            
            # get patient age
            age_years = int(df.loc[df.SessionLabel==ses_label]['AgeAtMRI'])
            # run gear
            inputs = {"antsct_output": zip_file}
            config = {"patient_age": age_years, "wthresholds": "0.0 0.5 1.0"}
            #analysis_id = gear.run(analysis_label='wscore_2021-11-04_PVS_WT', config=config, inputs=inputs, destination=session)
            
            run = 'ran_today'
            ants = analysis.label
            break # go back to sessions loop
        elif 'antsct' in analysis.label and analysis.job['state']=='complete' and analysis.files: # see if there's another ants run
            ants = analysis.label
            break # go back to sessions loop
        else:
            continue
            
    # Populate our data dict - remember that each key in the data dict must be updated
    # So that the length of our lists stays the same
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['run'].append(run)
    data_dict['ants'].append(ants)
    

105476x01x20190306x3T
continue
continue
123572x20190313x3T
continue
continue
continue
antsct_sub-123572_ses-123572x20190313x3T.zip


KeyError: 'AgeAtMRI'

In [ ]:
# Now create a data frame
df = pd.DataFrame.from_dict(data_dict)
df.head()

In [ ]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
time_fmt = '%m_%d_%Y-%H_%M_%S'
time_string = datetime.now().strftime(time_fmt)
csv_out = f'{gear.gear.name}_RunReport_{time_string}.csv'

df.to_csv(csv_out,index=False)

collection.upload_file(csv_out)